# Классификация покемонов часть 1
![alt text](https://giknutye.ru/wp-content/uploads/2016/08/chto-za-pokemon.jpg)


## Как загрузить колллаб на свой гугл диск

Чтобы иметь возможность запустить блоки кода в Google Colab, вам необходимо сохранить его на своем Google Disk.
Нажмите на вкладку File в левом верхнем углу, у вас откроется окно с различными пунктами, как видно на картинке ниже. Выберите пункт сохранить копию на диске (Save a copy in Drive).  

![alt text](https://sun2.is74.userapi.com/XPz5mAsxV4dq9DaQummKA-BUZO1zoa6yX5cj8w/Wdnxs_GHtv8.jpg)

У вас должна автоматически открыться новая вкладка с сохраненной копией Google Colab.

![alt text](https://sun1.is74.userapi.com/qheclDwr9RphL3yXKsGjyEbrYeYbCzp_cpM9cg/bMUYV8loGc4.jpg)

Нажмите на вкладку Edit и выберите пункт Notebook settings. 

![alt text](https://sun1.is74.userapi.com/vSOJHyGKJFg0SO_F6z9y3lwRSN5-e8Uc7CC9xQ/E-Q6EeGtmNQ.jpg)

Убедитесь, что у вас выбрано устройсво GPU.

![alt text](https://sun1.is74.userapi.com/5tLEM9N4hWVdbBNbBG88PaT8ByPR6va9hVpfdw/XcvHl6aKZs4.jpg)

---
## Что будем делать

На этом занятии мы построим и обучим сверточную нейронную сеть, которая будет способна распознать Пикачу и Бульбазавра. 


![alt text](https://images11.popmeh.ru/upload/img_cache/683/6836c73ba072928c5295be20b70f95f6_fitted_800x3000.jpg)

## Импорт библиотек
Для начала загрузим все необходимые библиотеки, они понадобятся для построения и обучения нейронной сети. 

Numpy -  это библиотека, добавляющая поддержку больших многомерных массивов и матриц, вместе с большой библиотекой высокоуровневых математических функций для операций с этими массивами.

Keras - открытая нейросетевая библиотека для оперативной работы с сетями глубокого обучения.

Matplotlib - библиотека для визуализации данных.



In [ ]:
import numpy as np
import math
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras import optimizers
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.preprocessing import image
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile
from google.colab import files
%matplotlib inline

## Загрузка данных
Загрузим набор изображений (датасет) для обучения сверточной нейронной сети.

Выберите иконку Files слева в вертикальном меню.

![alt text](https://sun1.is74.userapi.com/kFOVcPGnc82bkA5zoKykCt3j2q6xMa9SfmSTMg/zs5A0wjhmRs.jpg)

Нажмите на кнопку Upload, чтобы загрузить датасет **Pokemon_dataset_mini.zip** в Google Colab. [Ссылка на датасеты](https://drive.google.com/drive/folders/18vQ6tXnDm21mqn-02U-40jUvUaL1gYo9?usp=sharing)


![alt text](https://sun2.is74.userapi.com/fDp8RuYK5Mp-ULMxV0WtUfUub4HH4QBDsu_vSg/A16VSk2vl5s.jpg)

Поместить скачанный датасет необходимо в текущий каталог (где лежит папка sample_data)

In [ ]:
#!! не забудьте изменить название датасета, который будете загружать
fileName = 'Pokemon_dataset_mini.zip'
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

## Препроцессинг
Предварительно выполним предобработку полученного датасета, чтобы иметь возможность передать изображения нейроннной сети для ее обучения. 

Разобьём наш датасет на тренировочную, валидационную и тестовую выборки.

In [ ]:
# !! не забудьте обновить пути
train_path='Pokemon_dataset_mini/train/'
valid_path='Pokemon_dataset_mini/valid/'
test_path='Pokemon_dataset_mini/test/'

# !!измените названия классов (как указаны в директориях)
# !!например, если папки с именами car и bike --> classes=['car', 'bike']
train_batches=ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['pikachu','bulbasaur'], batch_size=20)
valid_batches=ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['pikachu','bulbasaur'], batch_size=10)
test_batches=ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['pikachu', 'bulbasaur'], batch_size=20)
label_map = (train_batches.class_indices)

In [ ]:
# Данная функция преобразует лейблы (индексы) в имена классов
def get_class_names(label):
  for name, index in label_map.items():
    if label==index:
      return name

## Визуализация датасета
Мы можем визуализировать изображения из датасета. Для этого запуститим два следующих блока. 


In [ ]:
def plots(ims, figsize=(12,6), rows=2, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims= np.array(ims).astype(np.uint8)
        if (ims.shape[-1]!=3):
            ims=ims.transpose((0,2,3,1))
    f=plt.figure(figsize=figsize)
    cols=len(ims)//rows if len(ims)%2==0 else len(ims)//rows +1
    for i in range(len(ims)):
        sp=f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(get_class_names(titles[i].argmax(axis=-1)), fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
imgs, labels=next(train_batches)
plots(imgs, titles=labels)

## Реализация сверточной нейронной сети
Далее построим сверточную нейронную сеть, используя библиотеку Keras.

Она будет включать в себя слои свертки Conv2D, полносвязанный слой и выходной.

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2), padding='same'),
    ##### задание 2 #####
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

## *ЗАДАНИЯ*
1. Во всех слоях замените функцию активации ***relu*** на ***sigmoid***, влияет ли это на результат?
2. Добавьте еще один слой свертки (Conv2D) после 
```
##### задание 2 #####
```
c количеством признаков равным 128 и слой пулинга (
MaxPooling2D) с теми же параметрами, что в предыдущей свертки.



## Компиляция нейронной сети
Скомпилируем нашу модель.

В качестве алгоритма оптимизации скорости обучения возьмем *Adam* с коэффициентом скорости 0.0001.

Функция потерь будет *categorical_crossentropy*, поскольку в нашем датасете несколько классов.

В качестве метрики (оценки производительности нашей модели) возьмем метрику *accuracy* (вычисляет как часто предсказания совпадают с метками).

In [ ]:
model.compile(optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

## *ЗАДАНИЯ*
1. Измените значение коэффициента скорости обучения (***lr***), допустимые пределы *от 0 до 0.5*, оцените результат.
2. Поменяйте алгоритм оптимизации с ***Adam*** на ***SGD***.

## Обучение нейронной сети
Запустим обучение нейронной сети.


In [ ]:
steps_epoch=math.ceil(train_batches.n/train_batches.batch_size)
valid_steps=math.ceil(valid_batches.n/valid_batches.batch_size)
model.fit_generator(train_batches, steps_per_epoch=steps_epoch, 
                   validation_data=valid_batches, validation_steps=valid_steps, epochs=20, verbose=2)

## Визуализация тестового набора
Отобразим подготовленный тестовый набор.

In [ ]:
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

## Тестирование нейронной сети на наборе
Выполним распознавание тестового набора

In [ ]:
model.predict_generator(test_batches, steps=1, verbose=0)

## Тестирование на одном изображении

Функция для предобработки тестового изображения перед подачей на вход нейронной сети.

In [ ]:
def preprocess_test_image(path):
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x_in=x/255.
  plt.imshow(x_in[0])                           
  plt.axis('off')
  plt.show()
  return x

Загрузка тестового изображения локально, из компьютера в Google Colab. 

In [ ]:
file_name=files.upload()

Запуск тестирования на входном изображении.

In [ ]:
img=preprocess_test_image(next(iter(file_name)))
pred=model.predict(img)
pred_classes = pred.argmax(axis=-1)
name=get_class_names(pred_classes)
print(name)

## Задания первой части практики



### Задание на изменение параметров нейронной сети
Вам необходимо будет выполнить задания после разделов ***Реализация сверточной нейронной сети*** и ***Компиляции нейронной сети***. 

После выполнения заданий, вам нужно будет прикрепить измененный Google Colab в курс на платформе. Для того чтобы скачать файл нажмите выберите File -> Download .ipynb.
![alt text](https://sun2.is74.userapi.com/EqGA1cIb3fITkfwnoV0nLgYgEM4b0xzvz94t2A/_ksVBmhhzWU.jpg)


### Задание на обучение нейронной сети на другом наборе данных

Вам представлено на выбор три различных датасета, каждый включает в себя по два класса. Необходимо будет обучить сверточную нейронную сеть на любом наборе данных.

[ссылка на датасеты](https://drive.google.com/drive/folders/1cQa73O8jHKVgL3kSYWPI4_36RImnWa9Q?usp=sharing)

Вам необходимо будет проделать те же шаги по загрузке датасета, что и в начале и изменить код в некоторых местах.

Блоки кода, в которых надо что-то поменять отмечены комментариями 

```
#!!какой-то текст
```



